## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
collections = db.list_collection_names()

In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
hygiene_20_count = establishments.count_documents(query)
print(f"Number of establishments with a hygiene score of 20: {hygiene_20_count}")
# Display the first document in the results using pprint
first_hygiene_20 = establishments.find_one(query)
pprint(first_hygiene_20)

Number of establishments with a hygiene score of 20: 0
None


In [7]:
# Convert the result to a Pandas DataFrame
results = list(establishments.find(query))
df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))
# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows in the DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    "LocalAuthorityName": {"$regex": "London", "$options": "i"},  # Match any Local Authority name containing 'London'
    "RatingValue": {"$gte": 4}
}
# Use count_documents to display the number of documents in the result
count = establishments.count_documents(query)
print("Number of establishments in London with RatingValue >= 4:", count)
# Display the first document in the results using pprint
first_document = establishments.find_one(query)
pprint(first_document)

Number of establishments in London with RatingValue >= 4: 0
None


In [9]:
# Convert the result to a Pandas DataFrame
results = list(establishments.find(query))
df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))
# Display the first 10 rows of the DataFrame
print(df.head(10))

Number of rows in the DataFrame: 0
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [10]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
penang_query = {"BusinessName": "Penang Flavours"}
penang_location = establishments.find_one(penang_query)
latitude = penang_location['geocode']['latitude']
longitude = penang_location['geocode']['longitude']

query = {
    "RatingValue": 5,
    "geocode.latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "geocode.longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search},
}
sort = [("scores.Hygiene", 1)]
limit = 5

# Print the results
results = list(establishments.find(query).sort(sort).limit(limit))
for result in results:
    pprint(result)

In [11]:
# Convert result to Pandas DataFrame


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [12]:
# Create a pipeline that:
pipeline = [
# 1. Matches establishments with a hygiene score of 0
    {"$match": {"scores.Hygiene": 0}},
# 2. Groups the matches by Local Authority
     {"$group": {
        "_id": "$LocalAuthorityName",
        "count": {"$sum": 1}
    }},
    
# 3. Sorts the matches from highest to lowest
 {"$sort": {"count": -1}}
]
results = list(establishments.aggregate(pipeline))
# Print the number of documents in the result
print("Number of Local Authorities with hygiene score of 0:", len(results))
# Print the first 10 results
print("Top 10 Local Authorities with highest number of hygiene score 0 establishments:")
for result in results[:10]:
    print(result)

Number of Local Authorities with hygiene score of 0: 0
Top 10 Local Authorities with highest number of hygiene score 0 establishments:


In [13]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)
# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))
# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []
